In [1]:
import numpy as np
import eli5
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# very messy please don't judge

In [3]:
# read the data and split
data = pd.read_csv("test_data.csv")
y = data["Status_code"]
data = data[["Height", "Weight", "Smoking", "Cholesterol"]]
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=4)

In [4]:
# fill missing values so it doesn't crash
fill_val = X_train.mean()
X_train = X_train.fillna(value=fill_val)
X_test = X_test.fillna(value=fill_val)

In [5]:
# fit and predict probabilities for case y=1
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
probs = model.predict_proba(X_test)[:, 1]

In [6]:
# any row that has probability less than 1 of being y=1
the_problem = X_test.iloc[np.where(probs < 0.5)[0][0]]
explanation = eli5.explain_prediction(model, the_problem, feature_names=["Height", "Weight", "Smoking", "Cholesterol"], targets=[1])
the_problem_but_in_dataframe = eli5.formatters.as_dataframe.format_as_dataframe(explanation)
print(the_problem_but_in_dataframe["weight"].sum())
explanation

-0.39000000000000007


Explanation(estimator="RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',\n            max_depth=None, max_features='auto', max_leaf_nodes=None,\n            min_impurity_decrease=0.0, min_impurity_split=None,\n            min_samples_leaf=1, min_samples_split=2,\n            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,\n            oob_score=False, random_state=None, verbose=0,\n            warm_start=False)", description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble (or a single tree for DecisionTreeClassifier).\nEach node of the tree has an output score, and contribution of a feature\non the decision path is how much the score changes from parent to child.\nWeights of all features sum to the output score or proba of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision path', is_regression=False, targets=[TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='Weight', weight=0.09209344762154294, std=None, value=187.0), FeatureWeight(feature='Height', weight=0.08509191307884552, std=None, value=67.75), FeatureWeight(feature='Smoking', weight=0.050084341460024766, std=None, value=20.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.6101655867530599, std=None, value=1.0), FeatureWeight(feature='Cholesterol', weight=-0.007104115407353424, std=None, value=200.0)], pos_remaining=0, neg_remaining=0), proba=0.39, score=None, weighted_spans=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None)